<a href="https://colab.research.google.com/github/thibaud-perrin/instruction-tuning/blob/main/notebooks/supervised_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Fine-Tuning SmolLM2 with SFTTrainer

In the following section, we will walk through the process of fine-tuning a model using the `SFTTrainer` from the `trl` library. The examples will demonstrate how to select and use datasets from the Hugging Face Hub for this purpose, offering varying levels of complexity:

- **🐢 Basic Example:** Fine-tune the model using the `HuggingFaceTB/smoltalk` dataset.  
- **🐕 Intermediate Example:** Fine-tune a code generation model with the `bigcode/the-stack-smol` dataset, focusing on the `data/python` subset.  

These examples will illustrate different approaches and levels of customization for supervised fine-tuning.


## Libraries

In [1]:
# Install the requirements in Google Colab
!pip install transformers datasets trl huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.4/293.4 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
# Authenticate to Hugging Face

from huggingface_hub import login
login()

# for convenience you can create an environment variable containing your hub token as HF_TOKEN

In [3]:
# Import necessary libraries
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available() else "cpu"
)

# Load the model and tokenizer
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(
    pretrained_model_name_or_path=model_name
).to(device)
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)

# Set up the chat format
model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)


config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/3.66k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/801k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

## Generate with the base model
Here we will try out the base model which does not have a chat template.

In [4]:
# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = model.generate(**inputs, max_new_tokens=100)
print("Before training:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Before training:
user
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a haiku about programming
Write a


## HuggingFaceTB/smoltalk

In [ ]:
# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-MyDataset"
finetune_tags = ["smol-course", "module_1"]

### Dataset Preparation
We will load a sample dataset and format it for training. The dataset should be structured with input-output pairs, where each input is a prompt and the output is the expected response from the model.

**TRL will format input messages based on the model's chat templates.** They need to be represented as a list of dictionaries with the keys: `role` and `content`.

In [5]:
# Load a sample dataset
from datasets import load_dataset

ds = load_dataset(path="HuggingFaceTB/smoltalk", name="everyday-conversations")

README.md:   0%|          | 0.00/9.25k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/52.6k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/119 [00:00<?, ? examples/s]

In [6]:
def process_dataset(sample):
    sample['chat_format_messages'] = tokenizer.apply_chat_template(sample['messages'], tokenize=False, add_generation_prompt=True)
    return sample

In [7]:
ds = ds.map(process_dataset)

Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

### Configuring the SFTTrainer
The `SFTTrainer` is configured with various parameters that control the training process. These include the number of training steps, batch size, learning rate, and evaluation strategy. Adjust these parameters based on your specific requirements and computational resources.

In [8]:
# Inspect the dataset structure and metadata
print(ds)

# Display dataset features
print(ds["train"].features)

# Check the number of examples in the train and test splits
print(f"Train split size: {len(ds['train'])}")
print(f"Test split size: {len(ds['test'])}")

# Peek at a few examples to understand the data format
print(ds["train"][0])

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages', 'chat_format_messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages', 'chat_format_messages'],
        num_rows: 119
    })
})
{'full_topic': Value(dtype='string', id=None), 'messages': [{'content': Value(dtype='string', id=None), 'role': Value(dtype='string', id=None)}], 'chat_format_messages': Value(dtype='string', id=None)}
Train split size: 2260
Test split size: 119
{'full_topic': 'Travel/Vacation destinations/Beach resorts', 'messages': [{'content': 'Hi there', 'role': 'user'}, {'content': 'Hello! How can I help you today?', 'role': 'assistant'}, {'content': "I'm looking for a beach resort for my next vacation. Can you recommend some popular ones?", 'role': 'user'}, {'content': "Some popular beach resorts include Maui in Hawaii, the Maldives, and the Bahamas. They're known for their beautiful beaches and crystal-clear waters.", 'role': 'assistant'}, {'c

In [9]:
num_epochs = 3
max_steps = len(ds["train"]) // 4 * num_epochs
print(f"Calculated max_steps: {max_steps}")

Calculated max_steps: 1695


In [10]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=max_steps,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=10,  # Frequency of logging training metrics
    save_steps=100,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=50,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model

)

# Pre-process: Extract the `chat_format_messages` column
train_dataset = ds["train"].map(lambda x: {"text": x["chat_format_messages"]}, remove_columns=ds["train"].column_names)
eval_dataset = ds["test"].map(lambda x: {"text": x["chat_format_messages"]}, remove_columns=ds["test"].column_names)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    eval_dataset=eval_dataset,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

<ipython-input-10-9ad76df164af>:23: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/2260 [00:00<?, ? examples/s]

Map:   0%|          | 0/119 [00:00<?, ? examples/s]

### Training the Model
With the trainer configured, we can now proceed to train the model. The training process will involve iterating over the dataset, computing the loss, and updating the model's parameters to minimize this loss.

In [11]:
import os

os.environ["WANDB_MODE"] = "disabled"

In [12]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
50,1.046200,1.139049
100,1.090500,1.103094
150,1.042800,1.075026
200,1.028800,1.060330
250,1.021600,1.051065
300,1.011700,1.041757
350,0.983500,1.035299
400,0.987700,1.032088
450,1.001600,1.023203
500,1.056200,1.013396


### Generating with the Fine-Tuned Model

In this section, we will demonstrate how to generate responses using the fine-tuned model. This process mirrors the approach used with the base model, showcasing how fine-tuning impacts the model's output.  


In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the fine-tuned model
fine_tuned_model_path = f"./{finetune_name}"
fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

In [18]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "Write a haiku about programming"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = fine_tuned_model.generate(**inputs, max_new_tokens=100)

In [24]:
# Decode and print the response
print("After fine-tuning:")
tokenizer.decode(outputs[0], skip_special_tokens=True)

"user\nWrite a haiku about programming\nassistant\nHello! How can I help you today? I'm a language model and I'm looking for help with programming. What programming language are you comfortable with? I'd be happy to help you learn. What programming language are you comfortable with? Python, Java, or something else? I can help you learn it. What programming language are you comfortable with? Python, Java, or something else? I can help you learn it. What programming language are you comfortable with? Python,"

## bigcode/the-stack-smol

In [4]:
# Set our name for the finetune to be saved &/ uploaded to
finetune_name = "SmolLM2-FT-stack-smol"
finetune_tags = ["stack-smol", "module_1"]

### Dataset Preparation

In [15]:
# Load a sample dataset
from datasets import load_dataset

ds = load_dataset(path="bigcode/the-stack-smol")

# Filter for Python files
ds = ds.filter(lambda example: example['lang'] == 'Python')

Resolving data files:   0%|          | 0/30 [00:00<?, ?it/s]

In [16]:
len(ds['train'])

10000

Thanks to this process_dataset function we will transform the dataset in a completion task dataset

In [17]:
import random

def process_dataset(sample):

    # Get the content and create a random starting portion for the user
    content = sample['content']
    content_lines = content.splitlines()

    # Randomly select a number of lines for the user message
    num_lines = random.randint(1, max(1, len(content_lines) - 1))  # At least 1 line, but not the full content
    user_message = "\n".join(content_lines[:num_lines])  # User gets the first `num_lines` lines

    assistant_message = f"```{sample['lang']}\n{content}\n```"

    # Build chat format
    messages = [
        {"role": "user", "content": user_message},
        {"role": "assistant", "content": assistant_message}
    ]

    # Apply template
    sample['chat_format_messages'] = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=False  # Avoid adding duplicate prompts
    )
    return sample
ds["train"] = ds["train"].shuffle(seed=42).select(range(5000))  # Randomly select first 5,000 after shuffle
ds = ds.map(process_dataset)

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

### Configuring the SFTTrainer

In [18]:
# Split the train dataset into train and test sets (e.g., 80% train, 20% test)
if "test" not in ds:
  ds = ds["train"].train_test_split(test_size=0.2, seed=42)

# Inspect the dataset structure and metadata
print(ds)

# Display dataset features
print(ds["train"].features)

# Check the number of examples in the train and test splits
print(f"Train split size: {len(ds['train'])}")
print(f"Test split size: {len(ds['test'])}")

# Peek at a few examples to understand the data format
print(ds["train"][0])
print(ds["test"][0])

DatasetDict({
    train: Dataset({
        features: ['content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'licenses', 'repository_name', 'path', 'size', 'lang', 'chat_format_messages'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['content', 'avg_line_length', 'max_line_length', 'alphanum_fraction', 'licenses', 'repository_name', 'path', 'size', 'lang', 'chat_format_messages'],
        num_rows: 1000
    })
})
{'content': Value(dtype='string', id=None), 'avg_line_length': Value(dtype='float64', id=None), 'max_line_length': Value(dtype='int64', id=None), 'alphanum_fraction': Value(dtype='float64', id=None), 'licenses': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'repository_name': Value(dtype='string', id=None), 'path': Value(dtype='string', id=None), 'size': Value(dtype='int64', id=None), 'lang': Value(dtype='string', id=None), 'chat_format_messages': Value(dtype='string', id=None)}
Train split size: 4000
Test split si

In [19]:
num_epochs = 3
train_size = len(ds["train"])

# Calculate max_steps as before
max_steps = train_size // 4 * num_epochs

# Determine eval_steps as a fraction of max_steps (e.g., every 10% of max_steps)
eval_steps = max_steps // 10  # Adjust the divisor for more/less frequent evaluations

# Determine save_steps as a fraction of max_steps (e.g., every 5% of max_steps)
save_steps = max_steps // 20  # Adjust the divisor for more/less frequent saves

# Determine logging_steps as a fraction of max_steps (e.g., every 2% of max_steps)
logging_steps = max_steps // 50  # Adjust the divisor for more/less frequent logs

print(f"Calculated max_steps: {max_steps}")
print(f"Calculated eval_steps: {eval_steps}")
print(f"Calculated save_steps: {save_steps}")
print(f"Calculated logging_steps: {logging_steps}")

Calculated max_steps: 3000
Calculated eval_steps: 300
Calculated save_steps: 150
Calculated logging_steps: 60


In [20]:
# Configure the SFTTrainer
sft_config = SFTConfig(
    output_dir="./sft_output",
    max_steps=max_steps,  # Adjust based on dataset size and desired training duration
    per_device_train_batch_size=4,  # Set according to your GPU memory capacity
    learning_rate=5e-5,  # Common starting point for fine-tuning
    logging_steps=logging_steps,  # Frequency of logging training metrics
    save_steps=save_steps,  # Frequency of saving model checkpoints
    evaluation_strategy="steps",  # Evaluate the model at regular intervals
    eval_steps=eval_steps,  # Frequency of evaluation
    use_mps_device=(
        True if device == "mps" else False
    ),  # Use MPS for mixed precision training
    hub_model_id=finetune_name,  # Set a unique name for your model

)

# Pre-process: Extract the `chat_format_messages` column
train_dataset = ds["train"].map(lambda x: {"text": x["chat_format_messages"]}, remove_columns=ds["train"].column_names)
eval_dataset = ds["test"].map(lambda x: {"text": x["chat_format_messages"]}, remove_columns=ds["test"].column_names)

# Initialize the SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=sft_config,
    train_dataset=train_dataset,
    tokenizer=tokenizer,
    eval_dataset=eval_dataset,
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

<ipython-input-20-50dacd0ca8a8>:23: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

### Training the Model

In [21]:
import os

os.environ["WANDB_MODE"] = "disabled"

In [22]:
# Train the model
trainer.train()

# Save the model
trainer.save_model(f"./{finetune_name}")

Step,Training Loss,Validation Loss
300,1.152800,1.178477
600,1.197900,1.166381
900,1.083100,1.156178
1200,1.077900,1.153828
1500,1.089300,1.149063
1800,1.003200,1.146037
2100,1.011400,1.152456
2400,0.973100,1.152455
2700,0.989900,1.150881
3000,0.958300,1.150217


### Generating with the Fine-Tuned Model

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the fine-tuned model
fine_tuned_model_path = f"./{finetune_name}"
fine_tuned_model = AutoModelForCausalLM.from_pretrained(fine_tuned_model_path).to(device)
tokenizer = AutoTokenizer.from_pretrained(fine_tuned_model_path)

In [24]:
# Test the fine-tuned model on the same prompt

# Let's test the base model before training
prompt = "# write a snake movement function in python"

# Format with template
messages = [{"role": "user", "content": prompt}]
formatted_prompt = tokenizer.apply_chat_template(messages, tokenize=False)

# Generate response
inputs = tokenizer(formatted_prompt, return_tensors="pt").to(device)
outputs = fine_tuned_model.generate(**inputs, max_new_tokens=100)

In [25]:
# Decode and print the response
print("After fine-tuning:")
tokenizer.decode(outputs[0], skip_special_tokens=True)

After fine-tuning:


'user\n# write a snake movement function in python\nassistant\n```Python\n# write a snake movement function in python\n\nimport random\nimport time\n\ndef snake_move(snake_x, snake_y, snake_length):\n    global speed\n    global direction\n    global direction_list\n    global direction_list_list\n    global snake_x, snake_y, snake_length\n    global direction_list, direction_list_list\n    global direction_list_list_list\n    global snake_x, snake_'